# Evaluating the consecutive k-mers c++ program output

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from eval_methods import *


## Reading the Data
Reading the data from the c++ program output into a pandas data frame.

In [2]:
# path to the dir that contains the output files from the cpp_code (filenames must end with .out)
# for example from the run: ./main -t 5 -i SRR23922262.fastq -s CAG
dir = '../out'


In [3]:
all_files = glob.glob(os.path.join(dir, "*.out"))
df = pd.concat((pd.read_csv(f, header=None) for f in all_files), ignore_index=True)
# seq. name: from the fasta or fastq file
# frame: [0, ..., k]
# repeat_representation: for example AG(GAC)_4 TGT
# score_type: run c++ main -h to see the available score types
# score: the score of the repeat_representation
# was_too_long: 1 if the if the input seq was longer than the configured max length, see c++ main -h
df.columns = ['seq_name', 'frame', 'repeat_representation', 'score_type', 'score', 'was_too_long']


In [4]:
# extract the values from the strings
df['frame'] = df['frame'].str.extract(r'frame[:\s]+(\d+)', expand=False).astype(int)
df['score_type'] = df["score_type"].str.extract(r'score_type[:\s]+(\w+)', expand=False)
df['score'] = df['score'].str.extract(r'score[:\s]+(\d+)', expand=False).astype(int)
df['was_too_long'] = df['was_too_long'].str.extract(
    r'seqlen too long[:\s]+(\w+)', expand=False).astype(int).astype(bool)
# creating new columns
# TODO: maybe dont use the +1 and adapt the pattern in seq_conforms_with_category accordingly
df["no_flanks"] = df["repeat_representation"].apply(lambda x: x[x.index("("):x.rfind(" ")+1])


## Grouping repeats into categories
Sine there are many repeats detected we want to categorize them. For this you can add categories to the list below. This
is a tow dimensional list. Every repeat category is a list of strings. For example (dropping "): `[(CAG), TAG, (CAG)]`
will add every sequence to this category if it contains a substring of this form `(CAG)_n TAG(CAG)_m`. When calling the
method `seq_conforms_with_category` to calculate the categories one can also pass a function that filters spurious
repeats. For example there might be the sequence `AAA(TTT)_2 ACGTAACCGGTT(GAC)_12 `, where the `(TTT)_2` repeat is probably
spurious and should be ignored. Here the method `is_spurious_by_max_repeat_len_and_min_distance` is applied. If all non
spurious repeats of a sequence are categorized and not in close proximity (separated by some minimum distance and therefore regarded
as independent) the sequence is flagged as `completely_defined`.

In [5]:
categories = [["(CAG)", "TAG", "(CAG)"],
              ["(CAG)", "CAA", "(CAG)"],
              ["(CAG)", "CCG", "(CAG)"],
              ["(CAG)", "CATCAGCAT", "(CAG)"],
              ["(CAG)"],
              ["(CAG)", "(CAA)"]]
# TODO also maybe with numbers as minimum requirements


In [6]:
max_repeat_len = 3
min_distance = 6


def f(x): return seq_conforms_with_category(x, categories, lambda seq,
                                            neighbour: is_spurious_by_max_repeat_len_and_min_distance(seq,
                                                                                                      max_repeat_len,
                                                                                                      min_distance,
                                                                                                      neighbour))


new_cols = df.apply(f, axis=1)
new_cols.columns = ["".join(c) for c in categories] + ["completely_defined"]
# deleting previously contained columns to avoid duplicates when concatenating
for column in new_cols.columns:
    if column in df.columns:
        df[column] = new_cols[column]
        del new_cols[column]

df = pd.concat([df, new_cols], axis=1)


## Viewing the data
Here you can have a look at the categorized data. If you still see some sequences for which the column
`completely_defined` is `False` consider adding new categories.

In [7]:
pd.set_option('display.max_colwidth', None)
print(f"From all {len(df)} samples {len(df[df['completely_defined']==1])} were categorizes completely")
df[:5]


From all 5734 samples 2849 were categorizes completely


,seq_name,frame,repeat_representation,score_type,score,was_too_long,no_flanks,(CAG)TAG(CAG),(CAG)CAA(CAG),(CAG)CCG(CAG),(CAG)CATCAGCAT(CAG),(CAG),(CAG)(CAA),completely_defined
0,@SRR23922262.1729.1 1729 length=151,2,TTCTCAGTGGACACACTCTTGGTCTCATAAGCAAGGAAGATTCC(CAG)_5 CCCCTTGTAACCATAGAAAATGCCAAGCCATGTATTCATCTTCCTGGAGCTGCAATGCTCCAGCTGGGGCAGAATAGAGACGTCAATATCTT,CAG,5,False,(CAG)_5,0,0,0,0,1,0,True
1,@SRR23922262.4951.2 4951 length=151,1,CAGCGACTAAAGG(CAG)_3 TGGCCGGTAGATTTT(CAG)_6 GGGGGAGGG(AGG)_2 (GGG)_2 GAAGGCCCACAAGGGTTGCGGCCCCCAGGG(GCT)_2 GGGGCTGGGCATAGTAAGGGGACGGCTGGGGCCGGTAGG,CAG,6,False,(CAG)_3 TGGCCGGTAGATTTT(CAG)_6 GGGGGAGGG(AGG)_2 (GGG)_2 GAAGGCCCACAAGGGTTGCGGCCCCCAGGG(GCT)_2,0,0,0,0,1,0,False
2,@SRR23922262.5224.2 5224 length=151,1,CCACCTGTCCTAGAAGCTG(CAG)_2 CAACGTATGCAGCTGGCCCGGAAACTG(CAG)_6 CAA(CAG)_4 CACCTTCTAGGACAGGTGGCAATC(CAG)_2 CAA(CAG)_3 GGTCCTGGAGTACAGACAAACAGA,CAG,9,False,(CAG)_2 CAACGTATGCAGCTGGCCCGGAAACTG(CAG)_6 CAA(CAG)_4 CACCTTCTAGGACAGGTGGCAATC(CAG)_2 CAA(CAG)_3,0,1,0,0,1,0,True
3,@SRR23922262.11181.2 11181 length=151,2,TGATC(CAG)_3 ATTGCCATC(CAC)_2 (CAG)_3 TTCCAGCACCGGCAGTCCCAGCTCCTTCACACAGCTACACACCTCCAGTTGGCG(CAG)_5 (CAA)_2 CAGCAA(CAG)_2 CAC(CAG)_3 CCGCAAGCCACCAC,CAG,5,False,(CAG)_3 ATTGCCATC(CAC)_2 (CAG)_3 TTCCAGCACCGGCAGTCCCAGCTCCTTCACACAGCTACACACCTCCAGTTGGCG(CAG)_5 (CAA)_2 CAGCAA(CAG)_2 CAC(CAG)_3,0,0,0,0,1,1,False
4,@SRR23922262.15112.2 15112 length=151,2,TGTAAGATGGGAATGCGTTCAGCCTCTATCATAGGACTGAAGACTGAATAAGCTAATATGTATCTAGTAAGTGTCTAATAAGTGTTAGT(TAT)_2 (CAG)_6 TGAAGGTCTTGCAGCAAGTCTCTCGGCTATCTAGCCGG,CAG,6,False,(TAT)_2 (CAG)_6,0,0,0,0,1,0,False


Look at the `m` most scoring sequences which are not yet completely categorized.

In [8]:
m = -1 # -1 to show almost all seqs. Adjust to positive integer to show smaller subset
sorted_by_score = df.sort_values(["score", "no_flanks"])
sorted_by_score = sorted_by_score[sorted_by_score["completely_defined"] == False]["no_flanks"][-m:]
color = Color_print_triplets()
for seq in list(sorted_by_score):
    print(color.color_triplets(seq, expand=True), "\\n")


AAAAAAAAGCAGCAGCAGCAGCAGCATGACAACTCAATGCAACTGGGATCCTGGACTGAATCCTGGACAAAAGGGTATTAT \n
AAAAAAAATCTGTGTATTAAAAAAAAAAAGCAGCAGCAGCAGCAG \n
AAAAAAAATCTGTGTATTAAAAAAAAAAAGCAGCAGCAGCAGCAGCTGAGTGTGATGGTGGGAGCCTGTAGTACCAGCTATTCAGGAGAATGAAGCAAGAGGATCACTTGAATCCAAGAGTTCAAATTAAACCTGTGTAAAAAA \n
AAAAAAATCTGTGTATTAAAAAAAAAAAAGCAGCAGCAGCAGCAG \n
AAAAAAATCTGTGTATTAAAAAAAAAAAAGCAGCAGCAGCAGCAG \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAG \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAGTACCACCAACAA \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAGTACCACCAACAA \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAGTACCACCAACAA \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAGTACCACCAACAA \n
AAAAAACCAACTAAATATCATCATCATCAGCAGCAGCAGCAGTACCACCAACAA \n
AAAAAAAAAAAGCAGCAGCAGCAGCAG \n
AAAAAAAAAAAGCAGCAGCAGCAGCAGCTGAGTGTGATGGTGGGAGCCTGTAGTACCAGCTATTCAGGAGAATGAAGCAAGAGGATCACTTGAATCCAAGAGTTCAAGTTCAGCCTGGGTAACATAGTAAAACACCACCAC \n
AAAAAAAAAAAGCAGCAGCAGCAGCAGCTGAGTGTGATGGTGGGAGCCTGTAGTACCAGCTATTCAGGAGAATGAAGCAAGAGGATCACTTGAATCCAAGAGTTCAAG

Inspect all sequences for a category:

In [9]:
category = "(CAG)TAG(CAG)"

df_of_category = df[(df["completely_defined"] == 1) & (df[category] == 1)][["no_flanks"]]
print(f"{len(df_of_category)} sequences were categorized as {category}:")
print(df_of_category)


108 sequences were categorized as (CAG)TAG(CAG):
                                                                                                        no_flanks
31                                                          (CAG)_8 TAG(CAG)_7 TAG(CAG)_10 TAG(CAG)_3 TAG(CAG)_3 
88                          (CAG)_4 TAG(CAG)_2 TAACAGTAGTAACGAGAG(AGA)_2 CTTTGATTCCACCTCTTC(CTC)_2 TTCCAC(TCC)_2 
104   (CAA)_2 CCCCACAGAGGACTTTAGGAGAAAGTTGCTGAAGGC(AGG)_2 GGACCCTAATAGGAGTATTCATAC(CAG)_8 TAG(CAG)_11 TAG(CAG)_5 
108                                                           (AGG)_2 GGACCCTAATAGGAGTACTCATAC(CAG)_8 TAG(CAG)_9 
109                                                           (AGG)_2 GGACCCTAATAGGAGTACTCATAC(CAG)_8 TAG(CAG)_9 
...                                                                                                           ...
5572                                                                                          (CAG)_3 TAG(CAG)_6 
5602                (TAG)_2 CAGTAG(CAG)